In [1]:
# 20/02/2023

In [2]:
import pandas as pd
import copy

In [3]:
def combine_tuning_results(tuning_results, output_address):
    """ Combines multiple tuning result DataFrames into one DataFrame"""
    
    assert type(tuning_results) == list
    assert len(tuning_results) > 1

    columns = set(tuning_results[0].columns)
    for tuning_result in tuning_results[1:]:
        assert set(tuning_result.columns) == columns
        

    combined_tuning_results = pd.DataFrame()

    for tuning_result in tuning_results:
        combined_tuning_results = combined_tuning_results.append(tuning_result)
    
    output_address_split = output_address.split('.csv')[0]

    tuning_result.to_csv(f'{output_address_split}.csv', index = False)





class YiLong:



    def __init__(self, type):
        
        # check correct input
        assert type == 'Classification' or type == 'Regression' or 'GLM Regression'

        self.clf_type = type
        self._initialise_objects() # Initialise objects
        print(f'YiLong Initialised to analyse {self.clf_type}')



    def _initialise_objects(self):
        """ Helper to initialise objects """

        self.tuning_result = None
        self.hyperparameters = None
        self._seed = 18861201

        self.regression_extra_output_columns = ['Train r2', 'Val r2', 'Test r2', 
            'Train RMSE', 'Val RMSE', 'Test RMSE', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Time']
        self.classification_extra_output_columns = ['Train accu', 'Val accu', 'Test accu', 
            'Train balanced_accu', 'Val balanced_accu', 'Test balanced_accu', 'Train f1', 'Val f1', 'Test f1', 
            'Train precision', 'Val precision', 'Test precision', 'Train recall', 'Val recall', 'Test recall', 'Time']
        self.GLM_Regression_extra_output_columns = ['Train deviance', 'Val deviance', 'Test deviance', 'Time']

        self.discard_columns = None

        self.tuning_result = None
        self.hyperparameters = None

    

    def read_tuning_result(self, address, extra_to_discard_columns = None):
        """ Read in Tuning Result """

        if extra_to_discard_columns is not None:
            assert type(extra_to_discard_columns) == list

        self.tuning_result = pd.read_csv(address)

        print(f'Successfully read in tuning result, with {len(self.tuning_result)} columns')

        # get list of hyperparameters by taking what is not in the extra_output_columns
        if self.clf_type == 'Classification':
            self.discard_columns = copy.deepcopy(self.classification_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'Regression':
            self.discard_columns = copy.deepcopy(self.regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'GLM Regression':
            self.discard_columns = copy.deepcopy(self.GLM_Regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)

        self.hyperparameters = [col for col in self.tuning_result.columns if col not in self.discard_columns]



    def read_sorted_full_df(self, interested_statistic = None, ascending = False):
        """ View dataframe sorted in reverse in terms of validation score """
        
        assert type(ascending) == bool

        if self.tuning_result is None:
            print('Please run read_tuning_result() first')
            return
        
        if interested_statistic is not None:
            if self.clf_type == 'Regression':
                if interested_statistic not in self.regression_extra_output_columns:
                    print('Statistic not valid for a Regression Model')
                    return

            elif self.clf_type == 'Classification':
                if interested_statistic not in self.classification_extra_output_columns:
                    print('Statistic not valid for a Classification Model')
                    return
            
            elif self.clf_type == 'GLM Regression':
                if interested_statistic not in self.GLM_Regression_extra_output_columns:
                    print('Statistic not valid for a GLM Regression Model')
                    return
        
        

        if len(self.tuning_result) < 60:
            length = len(self.tuning_result)
        else:
            length = 60


        if self.clf_type =='Regression':
            if interested_statistic == None:
                interested_statistic = 'Val r2'

            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)

        elif self.clf_type =='Classification':
            if interested_statistic == None:
                interested_statistic = 'Val accu'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)
        
        elif self.clf_type =='GLM Regression':
            if interested_statistic == None:
                interested_statistic = 'Val deviance'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)

        

        sorted_tuning_results.index = range(len(sorted_tuning_results))
        best_hyperparameter_combination = {hyperparameter:sorted_tuning_results.iloc[0][hyperparameter] for hyperparameter in self.hyperparameters}

        # change layout of output
        out_best_hyperparameter_combination = [copy.deepcopy(best_hyperparameter_combination)]
        if 'features' in out_best_hyperparameter_combination[0]:
            out_best_hyperparameter_combination.extend([out_best_hyperparameter_combination[0]['features'], {'feature combo ningxiang score':out_best_hyperparameter_combination[0]['feature combo ningxiang score']}])
            del out_best_hyperparameter_combination[0]['features']
            del out_best_hyperparameter_combination[0]['feature combo ningxiang score']

        print('Best hyperameter combination:', out_best_hyperparameter_combination, '\n')

        print(f'Highest {length}')
        display(sorted_tuning_results.head(length))
        print(f'Lowest {length}')
        display(sorted_tuning_results.tail(length))


        return out_best_hyperparameter_combination 



    def read_mean_val_scores(self):
        """ View the means of evaluation metrics for combinations containing each individual value of a hyperparameter – for each hyperparameter """

        for col in self.hyperparameters: # for each hyperparameter
            
            if col == 'features': # report NingXiang score only
                    continue

            print('\nHYPERPARAMETER:', col.upper())

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            validation_score_df = pd.DataFrame()
            for value in hyperparameter_values: # for each value in the hyperparameter

                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                # get means
                if self.clf_type == 'Classification':
                    tmp_df_mean = tmp_df[self.classification_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'Regression':
                    tmp_df_mean = tmp_df[self.regression_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'GLM Regression':
                    tmp_df_mean = tmp_df[self.GLM_Regression_extra_output_columns[:-1]].mean().T

                # get number of observations in this group
                tmp_df_mean['n'] = len(tmp_df)

                # append to this temporary dataframe
                validation_score_df[f'{value}'] = tmp_df_mean

            display(validation_score_df)
    


    def read_grouped_scores(self):
        """ View all evaluation metrics for combinations grouped by containing each individual value of a hyperparameter – for each hyperparameter 
        If any of the individual values of a hyperparameter exceeds 60, then sample down to 60 without replacement """

        for col in self.hyperparameters: # for each hyperparameter

            if col == 'features': # report NingXiang score only
                    continue

            print('\nHYPERPARAMETER:', col.upper())

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            for value in hyperparameter_values: # for each value in the hyperparameter

                print(f'{col} Value:', value)
                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                if len(tmp_df) < 60:
                    length = len(tmp_df)
                else:
                    length = 60

                display(tmp_df.sample(length, replace=False, random_state=self._seed))

In [4]:
yilong = YiLong('Regression')

YiLong Initialised to analyse Regression


In [5]:
yilong.read_tuning_result('../models/tuning/testtt.csv')

Successfully read in tuning result, with 358 columns


In [6]:
yilong.read_sorted_full_df()

Best hyperameter combination: [{'max_bins': 67.0, 'learning_rate': 0.1}, "['Disposals BTRS', 'Score Involvements BTRS', 'Winloss', 'Disposals OTRS', 'Behind Assists BTRS', 'Tackles BTRS', 'Kicks BTRS', 'Contested Marks OTRS', 'Contested Possessions BTRS', 'Hitouts BTRS', 'Marks BTRS', 'Inside 50s BTRS']", {'feature combo ningxiang score': 0.5980934780327644}] 

Highest 60


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
0,67.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.405094,0.394916,0.378625,0.194096,0.195740,0.198462,1.726761e+14,1.767109e+14,1.772789e+14,4.84,0.598093
1,127.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.405912,0.393988,0.378607,0.193963,0.195891,0.198465,1.725436e+14,1.769116e+14,1.773173e+14,3.02,0.598093
2,130.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.405989,0.393959,0.378515,0.193950,0.195895,0.198479,1.725414e+14,1.769184e+14,1.773333e+14,3.07,0.598093
3,255.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.406034,0.393958,0.378628,0.193943,0.195895,0.198461,1.725238e+14,1.769196e+14,1.773179e+14,5.37,0.598093
4,256.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.406034,0.393958,0.378628,0.193943,0.195895,0.198461,1.725238e+14,1.769196e+14,1.773179e+14,3.07,0.598093
5,193.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.406034,0.393958,0.378628,0.193943,0.195895,0.198461,1.725238e+14,1.769196e+14,1.773179e+14,4.47,0.598093
6,128.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.405900,0.393948,0.378552,0.193965,0.195897,0.198473,1.725546e+14,1.769267e+14,1.773302e+14,3.74,0.598093
7,67.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.400679,0.393775,0.377663,0.194815,0.195925,0.198615,1.742692e+14,1.777728e+14,1.782254e+14,7.79,0.598093
8,52.0,0.042170,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.399202,0.393402,0.374532,0.195055,0.195985,0.199114,1.745498e+14,1.781176e+14,1.789030e+14,3.56,0.558760
9,36.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.397085,0.392328,0.378505,0.195399,0.196159,0.198481,1.739254e+14,1.770833e+14,1.775269e+14,3.96,0.598093


Lowest 60


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
298,193.0,3.162278,"['Disposals BTRS', 'Score Involvements BTRS', ...",-3.041218e+02,-3.042125e+02,-3.037171e+02,4.395718,4.396165,4.394892,9.170199e+14,9.206369e+14,9.142266e+14,3.75,0.761016
299,4.0,100.000000,['Disposals BTRS'],-1.349594e+05,-1.349729e+05,-1.348318e+05,92.447740,92.448038,92.448026,2.710464e+16,2.713944e+16,2.713011e+16,0.13,0.231460
300,256.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.23,0.231460
301,128.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.21,0.231460
302,130.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.14,0.231460
303,127.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.14,0.231460
304,255.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.13,0.231460
305,193.0,100.000000,['Disposals BTRS'],-1.365835e+05,-1.365950e+05,-1.364654e+05,93.002332,93.001896,93.006379,2.213758e+16,2.220959e+16,2.218610e+16,0.22,0.231460
306,67.0,100.000000,['Disposals BTRS'],-1.365839e+05,-1.365952e+05,-1.364656e+05,93.002465,93.001967,93.006443,2.213761e+16,2.221086e+16,2.218576e+16,0.13,0.231460
307,96.0,17.782794,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.104358e+06,-1.104252e+06,-1.103280e+06,264.452946,264.427872,264.449547,6.306425e+16,6.310126e+16,6.299476e+16,1.80,0.490855


[{'max_bins': 67.0, 'learning_rate': 0.1},
 "['Disposals BTRS', 'Score Involvements BTRS', 'Winloss', 'Disposals OTRS', 'Behind Assists BTRS', 'Tackles BTRS', 'Kicks BTRS', 'Contested Marks OTRS', 'Contested Possessions BTRS', 'Hitouts BTRS', 'Marks BTRS', 'Inside 50s BTRS']",
 {'feature combo ningxiang score': 0.5980934780327644}]

In [7]:
yilong.read_mean_val_scores()


HYPERPARAMETER: MAX_BINS


,4.0,20.0,36.0,52.0,67.0,96.0,127.0,128.0,130.0,162.0,193.0,224.0,255.0,256.0
Train r2,-1.503144e+08,3.618324e-01,-1.193424e+05,3.734357e-01,-1.304674e+08,-2.760893e+05,-2.139673e+08,-2.139673e+08,-2.388016e+08,-2.760905e+05,-2.139673e+08,-2.760905e+05,-2.139673e+08,-2.388016e+08
Val r2,-1.503279e+08,3.628506e-01,-1.193978e+05,3.717434e-01,-1.304557e+08,-2.760628e+05,-2.139481e+08,-2.139481e+08,-2.387801e+08,-2.760638e+05,-2.139481e+08,-2.760638e+05,-2.139481e+08,-2.387801e+08
Test r2,-1.501720e+08,3.525919e-01,-1.192361e+05,3.555410e-01,-1.303407e+08,-2.758197e+05,-2.137595e+08,-2.137595e+08,-2.385697e+08,-2.758209e+05,-2.137595e+08,-2.758209e+05,-2.137595e+08,-2.385697e+08
Train RMSE,1.042745e+03,2.010061e-01,2.877494e+01,1.991623e-01,9.007034e+02,6.626087e+01,1.477027e+03,1.477027e+03,1.647446e+03,6.626099e+01,1.477027e+03,6.626099e+01,1.477027e+03,1.647446e+03
Val RMSE,1.042743e+03,2.008434e-01,2.878017e+01,1.994324e-01,9.006216e+02,6.625577e+01,1.476893e+03,1.476893e+03,1.647296e+03,6.625591e+01,1.476893e+03,6.625591e+01,1.476893e+03,1.647296e+03
Test RMSE,1.042749e+03,2.025519e-01,2.877802e+01,2.020936e-01,9.006965e+02,6.626352e+01,1.477014e+03,1.477014e+03,1.647431e+03,6.626368e+01,1.477014e+03,6.626368e+01,1.477014e+03,1.647431e+03
Train MAPE,3.030209e+17,1.846056e+14,6.995835e+15,1.840562e+14,2.153645e+17,1.589994e+16,3.530616e+17,3.530616e+17,3.937943e+17,1.589911e+16,3.530616e+17,1.589911e+16,3.530616e+17,3.937943e+17
Val MAPE,3.034001e+17,1.861090e+14,6.981636e+15,1.862336e+14,2.155202e+17,1.591195e+16,3.533147e+17,3.533147e+17,3.940760e+17,1.591116e+16,3.533147e+17,1.591116e+16,3.533147e+17,3.940760e+17
Test MAPE,3.032910e+17,1.862649e+14,7.009794e+15,1.869191e+14,2.151575e+17,1.588591e+16,3.527196e+17,3.527196e+17,3.934122e+17,1.588506e+16,3.527196e+17,1.588506e+16,3.527196e+17,3.934122e+17
n,4.400000e+01,1.600000e+01,3.700000e+01,1.600000e+01,4.100000e+01,1.600000e+01,2.500000e+01,2.500000e+01,2.800000e+01,1.600000e+01,2.500000e+01,1.600000e+01,2.500000e+01,2.800000e+01



HYPERPARAMETER: LEARNING_RATE


,0.0001,0.0002371373705661,0.0005623413251903,0.0013335214321633,0.0031622776601683,0.0074989420933245,0.0177827941003892,0.0421696503428582,0.1,0.5623413251903491,3.1622776601683795,17.78279410038923,100.0
Train r2,-5.781653e-01,3.444594e-01,3.411114e-01,3.680066e-01,3.551476e-01,3.769805e-01,3.668053e-01,3.810896e-01,3.673501e-01,3.986650e-01,-2.559806e+02,-1.104251e+06,-1.086805e+09
Val r2,-5.754827e-01,3.497716e-01,3.426941e-01,3.691831e-01,3.500741e-01,3.744819e-01,3.587031e-01,3.757514e-01,3.536494e-01,3.823114e-01,-2.559889e+02,-1.104299e+06,-1.086735e+09
Test r2,-5.836332e-01,3.375919e-01,3.271606e-01,3.561779e-01,3.365993e-01,3.607774e-01,3.421146e-01,3.617186e-01,3.412593e-01,3.624893e-01,-2.556571e+02,-1.103196e+06,-1.085753e+09
Train RMSE,2.763289e-01,2.037353e-01,2.039605e-01,2.000400e-01,2.018180e-01,1.986111e-01,1.999737e-01,1.979502e-01,1.998755e-01,1.951361e-01,3.922881e+00,2.644401e+02,7.501792e+03
Val RMSE,2.759092e-01,2.028984e-01,2.037047e-01,1.998469e-01,2.026272e-01,1.990050e-01,2.012704e-01,1.988015e-01,2.020808e-01,1.977678e-01,3.922789e+00,2.644335e+02,7.501201e+03
Test RMSE,2.774152e-01,2.048949e-01,2.062486e-01,2.019990e-01,2.048668e-01,2.012739e-01,2.040003e-01,2.011233e-01,2.041501e-01,2.010150e-01,3.922333e+00,2.644395e+02,7.501729e+03
Train MAPE,1.856228e+14,1.965740e+14,1.914105e+14,1.844835e+14,1.859213e+14,1.790540e+14,1.822772e+14,1.772122e+14,1.817607e+14,1.732978e+14,8.992850e+14,6.309529e+16,1.847819e+18
Val MAPE,1.861533e+14,1.970822e+14,1.927648e+14,1.860536e+14,1.883971e+14,1.814526e+14,1.855917e+14,1.800968e+14,1.857274e+14,1.781515e+14,9.018416e+14,6.308693e+16,1.849299e+18
Test MAPE,1.867187e+14,1.977481e+14,1.935333e+14,1.864679e+14,1.888128e+14,1.817524e+14,1.862625e+14,1.803994e+14,1.858662e+14,1.789918e+14,8.979245e+14,6.307013e+16,1.846596e+18
n,5.200000e+01,8.000000e+00,2.900000e+01,8.000000e+00,4.900000e+01,8.000000e+00,2.900000e+01,8.000000e+00,5.200000e+01,1.600000e+01,4.000000e+01,1.600000e+01,4.300000e+01



HYPERPARAMETER: FEATURE COMBO NINGXIANG SCORE


,0.2314600598811033,0.4077702362529444,0.4908548540647465,0.5587601652774793,0.5980934780327644,0.6505248469895428,0.6835133008876983,0.730940920898561,0.7610156369458968,0.8310076610609045,0.894973593857727,0.9068508953197888,0.9607867003479292,1.0
Train r2,-2.227759e+04,3.409771e-01,-1.523102e+05,3.773541e-01,-2.180320e+08,3.769088e-01,-1.523102e+05,3.752961e-01,-2.180320e+08,-2.760624e+05,-2.670892e+08,-2.670892e+08,-2.760624e+05,-4.441402e+08
Val r2,-2.227951e+04,3.437090e-01,-1.523168e+05,3.778331e-01,-2.180172e+08,3.749062e-01,-1.523168e+05,3.727396e-01,-2.180172e+08,-2.760745e+05,-2.670711e+08,-2.670711e+08,-2.760745e+05,-4.441267e+08
Test r2,-2.225811e+04,3.250449e-01,-1.521647e+05,3.632180e-01,-2.178208e+08,3.649708e-01,-1.521647e+05,3.630321e-01,-2.178208e+08,-2.757988e+05,-2.668305e+08,-2.668305e+08,-2.757988e+05,-4.437124e+08
Train RMSE,1.572816e+01,2.042775e-01,3.664773e+01,1.985520e-01,1.502262e+03,1.986242e-01,3.664554e+01,1.988829e-01,1.502262e+03,6.625725e+01,1.840226e+03,1.840226e+03,6.625700e+01,3.062036e+03
Val RMSE,1.572794e+01,2.038469e-01,3.664632e+01,1.984731e-01,1.502141e+03,1.989428e-01,3.664560e+01,1.992890e-01,1.502142e+03,6.625717e+01,1.840079e+03,1.840079e+03,6.625736e+01,3.061847e+03
Test RMSE,1.572877e+01,2.068364e-01,3.665130e+01,2.008989e-01,1.502250e+03,2.006246e-01,3.664839e+01,2.009311e-01,1.502251e+03,6.626034e+01,1.840212e+03,1.840212e+03,6.626058e+01,3.062019e+03
Train MAPE,3.958028e+15,1.908069e+14,8.863074e+15,1.819272e+14,3.687310e+17,1.822283e+14,8.859968e+15,1.823613e+14,3.687315e+17,1.590679e+16,4.516544e+17,4.516545e+17,1.590623e+16,7.842871e+17
Val MAPE,3.969174e+15,1.917385e+14,8.863483e+15,1.839599e+14,3.690225e+17,1.844108e+14,8.861426e+15,1.845761e+14,3.690233e+17,1.590762e+16,4.520113e+17,4.520115e+17,1.590749e+16,7.850024e+17
Test MAPE,3.965825e+15,1.930735e+14,8.863552e+15,1.847379e+14,3.684725e+17,1.842709e+14,8.859295e+15,1.842857e+14,3.684731e+17,1.590340e+16,4.513375e+17,4.513375e+17,1.590320e+16,7.840712e+17
n,4.900000e+01,8.000000e+00,2.900000e+01,8.000000e+00,4.900000e+01,8.000000e+00,2.900000e+01,8.000000e+00,4.900000e+01,1.600000e+01,4.000000e+01,4.000000e+01,1.600000e+01,9.000000e+00


In [8]:
yilong.read_grouped_scores()


HYPERPARAMETER: MAX_BINS
max_bins Value: 4.0


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
3,4.0,100.000000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.322739e+09,-1.322858e+09,-1.321486e+09,9152.301752,9152.283531,9152.318838,2.658791e+18,2.662108e+18,2.661142e+18,6.59,1.000000
307,4.0,0.003162,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.662650e-01,3.558930e-01,3.438530e-01,0.200330,0.201954,0.203939,1.813648e+14,1.849007e+14,1.846252e+14,22.04,0.683513
35,4.0,0.000100,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.129110e-01,3.103900e-01,3.044090e-01,0.208593,0.208965,0.209979,2.092867e+14,2.103959e+14,2.106112e+14,96.92,0.598093
2,4.0,100.000000,['Disposals BTRS'],-1.349594e+05,-1.349729e+05,-1.348318e+05,92.447740,92.448038,92.448026,2.710464e+16,2.713944e+16,2.713011e+16,0.13,0.231460
327,4.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.694460e-01,3.682490e-01,3.390760e-01,0.199827,0.200007,0.204680,1.812743e+14,1.838113e+14,1.870454e+14,2.50,0.490855
10,4.0,0.100000,['Disposals BTRS'],4.955100e-02,4.692100e-02,4.977800e-02,0.245334,0.245662,0.245421,2.707379e+14,2.710855e+14,2.709924e+14,0.18,0.231460
37,4.0,0.003162,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.739390e-01,3.582550e-01,3.515520e-01,0.199114,0.201583,0.202739,1.806381e+14,1.848578e+14,1.839335e+14,16.92,0.598093
78,4.0,0.000100,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.069910e-01,3.066190e-01,2.968520e-01,0.209490,0.209536,0.211117,2.107991e+14,2.116023e+14,2.122948e+14,113.31,0.906851
274,4.0,0.000562,['Disposals BTRS'],3.200000e-05,-2.456000e-03,4.150000e-04,0.251644,0.251945,0.251715,2.547687e+14,2.550958e+14,2.550081e+14,6.30,0.231460
324,4.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.809430e-01,3.593110e-01,3.544790e-01,0.197997,0.201417,0.202281,1.768084e+14,1.820969e+14,1.810129e+14,9.08,0.598093


max_bins Value: 20.0


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
309,20.0,0.001334,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.371046,0.372087,0.365919,0.199573,0.199399,0.200481,1.824435e+14,1.840794e+14,1.834918e+14,19.79,0.650525
302,20.0,0.000237,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.350285,0.355075,0.349138,0.202840,0.202082,0.203116,1.944166e+14,1.950153e+14,1.948737e+14,53.71,0.650525
350,20.0,0.042170,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.381194,0.374722,0.369825,0.197957,0.198980,0.199862,1.751294e+14,1.778595e+14,1.767986e+14,5.37,0.730941
332,20.0,0.042170,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.384768,0.385016,0.370919,0.197384,0.197335,0.199689,1.748346e+14,1.772168e+14,1.780991e+14,5.40,0.558760
326,20.0,0.007499,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.382665,0.379074,0.371619,0.197721,0.198286,0.199577,1.761391e+14,1.789224e+14,1.787167e+14,10.28,0.558760
323,20.0,0.007499,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.344076,0.347249,0.327599,0.203807,0.203304,0.206450,1.863112e+14,1.873030e+14,1.884845e+14,3.34,0.407770
345,20.0,0.007499,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.375227,0.373308,0.368273,0.198909,0.199205,0.200108,1.777204e+14,1.798293e+14,1.788320e+14,9.70,0.730941
287,20.0,0.001334,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.339152,0.344178,0.325636,0.204571,0.203782,0.206751,1.914164e+14,1.918360e+14,1.931675e+14,11.26,0.407770
289,20.0,0.001334,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.371080,0.371161,0.363316,0.199568,0.199546,0.200892,1.820328e+14,1.836313e+14,1.838936e+14,17.42,0.558760
283,20.0,0.000237,"['Disposals BTRS', 'Score Involvements BTRS', ...",0.347723,0.353595,0.341944,0.203240,0.202314,0.204236,1.948361e+14,1.952944e+14,1.960826e+14,43.61,0.558760


max_bins Value: 36.0


...

,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
198,193.0,0.000100,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.332520e-01,3.358170e-01,3.244440e-01,0.205482,0.205077,0.206933,2.056286e+14,2.064051e+14,2.070508e+14,102.26,0.906851
259,193.0,100.000000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.337261e+09,-1.337141e+09,-1.335963e+09,9202.405010,9201.560484,9202.311794,2.199326e+18,2.200876e+18,2.197168e+18,5.51,0.906851
81,67.0,0.003162,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.916820e-01,3.801250e-01,3.644910e-01,0.196272,0.198118,0.200706,1.768398e+14,1.805590e+14,1.807199e+14,23.65,0.906851
258,130.0,100.000000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.337261e+09,-1.337141e+09,-1.335963e+09,9202.405010,9201.560484,9202.311794,2.199326e+18,2.200876e+18,2.197168e+18,7.69,0.906851
103,127.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.070930e-01,3.820580e-01,3.711730e-01,0.193770,0.197809,0.199648,1.715579e+14,1.776972e+14,1.768614e+14,7.16,0.906851
86,4.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.578250e-01,3.479560e-01,3.295530e-01,0.201660,0.203194,0.206149,1.785457e+14,1.820322e+14,1.828913e+14,9.39,0.906851
87,67.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.059970e-01,3.829720e-01,3.705090e-01,0.193949,0.197663,0.199754,1.716535e+14,1.773938e+14,1.768069e+14,6.37,0.906851
205,193.0,0.100000,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.070060e-01,3.821320e-01,3.710700e-01,0.193784,0.197797,0.199665,1.715496e+14,1.776631e+14,1.768451e+14,6.36,0.906851
97,127.0,0.003162,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.918740e-01,3.798100e-01,3.644960e-01,0.196241,0.198169,0.200705,1.768673e+14,1.807129e+14,1.808838e+14,24.97,0.906851
78,4.0,0.000100,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.069910e-01,3.066190e-01,2.968520e-01,0.209490,0.209536,0.211117,2.107991e+14,2.116023e+14,2.122948e+14,113.31,0.906851


feature combo ningxiang score Value: 0.9607867003479292


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
150,96.0,0.562341,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.072000e-01,3.786280e-01,3.675530e-01,0.193752,0.198358,0.200222,1.705420e+14,1.769560e+14,1.760684e+14,8.22,0.960787
138,36.0,0.562341,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.006300e-01,3.786560e-01,3.650900e-01,0.194823,0.198353,0.200611,1.714998e+14,1.767145e+14,1.763834e+14,6.44,0.960787
272,224.0,17.782794,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.104363e+06,-1.104257e+06,-1.103285e+06,264.453505,264.428378,264.450127,6.306100e+16,6.309802e+16,6.299133e+16,8.21,0.960787
222,224.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.037450e-01,3.807550e-01,3.686890e-01,0.194316,0.198018,0.200042,1.727827e+14,1.784833e+14,1.779887e+14,15.00,0.960787
206,162.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.037450e-01,3.807550e-01,3.686890e-01,0.194316,0.198018,0.200042,1.727827e+14,1.784833e+14,1.779887e+14,15.06,0.960787
200,162.0,0.000562,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.744020e-01,3.704770e-01,3.540820e-01,0.199040,0.199654,0.202343,1.854482e+14,1.877092e+14,1.883975e+14,55.23,0.960787
260,162.0,17.782794,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.104363e+06,-1.104257e+06,-1.103285e+06,264.453505,264.428378,264.450127,6.306100e+16,6.309802e+16,6.299133e+16,8.63,0.960787
98,96.0,0.000562,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.744740e-01,3.705720e-01,3.541670e-01,0.199029,0.199639,0.202330,1.854007e+14,1.876484e+14,1.883439e+14,52.77,0.960787
104,96.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.032370e-01,3.808790e-01,3.686280e-01,0.194399,0.197998,0.200052,1.728792e+14,1.784553e+14,1.779988e+14,14.48,0.960787
88,36.0,0.017783,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.944770e-01,3.824590e-01,3.677300e-01,0.195821,0.197745,0.200194,1.741735e+14,1.779875e+14,1.781154e+14,14.33,0.960787


feature combo ningxiang score Value: 1.0


,max_bins,learning_rate,features,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time,feature combo ningxiang score
18,130.0,0.1000,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.084980e-01,3.807870e-01,3.703180e-01,0.193540,0.198013,0.199784,1.711790e+14,1.775207e+14,1.767122e+14,9.18,1.0
5,256.0,0.0001,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.309970e-01,3.334430e-01,3.205410e-01,0.205829,0.205443,0.207530,2.053612e+14,2.061300e+14,2.070484e+14,116.08,1.0
7,256.0,100.0000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.337261e+09,-1.337141e+09,-1.335963e+09,9202.405010,9201.560484,9202.311794,2.199326e+18,2.200876e+18,2.197168e+18,7.82,1.0
3,4.0,100.0000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.322739e+09,-1.322858e+09,-1.321486e+09,9152.301752,9152.283531,9152.318838,2.658791e+18,2.662108e+18,2.661142e+18,6.59,1.0
16,4.0,0.1000,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.577060e-01,3.466680e-01,3.302360e-01,0.201679,0.203395,0.206044,1.783672e+14,1.821868e+14,1.827125e+14,10.60,1.0
1,4.0,0.0001,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.079620e-01,3.076920e-01,2.975760e-01,0.209343,0.209374,0.211008,2.103223e+14,2.112461e+14,2.118431e+14,119.50,1.0
30,256.0,0.1000,"['Disposals BTRS', 'Score Involvements BTRS', ...",4.082110e-01,3.809850e-01,3.704040e-01,0.193587,0.197981,0.199770,1.712697e+14,1.775309e+14,1.767455e+14,8.14,1.0
24,130.0,100.0000,"['Disposals BTRS', 'Score Involvements BTRS', ...",-1.337261e+09,-1.337141e+09,-1.335963e+09,9202.405010,9201.560484,9202.311794,2.199326e+18,2.200876e+18,2.197168e+18,8.58,1.0
17,130.0,0.0001,"['Disposals BTRS', 'Score Involvements BTRS', ...",3.309970e-01,3.334430e-01,3.205400e-01,0.205829,0.205443,0.207530,2.053613e+14,2.061300e+14,2.070485e+14,115.32,1.0
